# PMML

El objetivo del ejercicio es tomar el codigo anterior y modificarlo para que el resultado de la API sea más útil. Es decir, no solamente cambiar el resultado numérico a clases nuevamente, sino también la fuente para poder pasarle la data sin normalizar (como viene en el dataset).

No hay una única manera de hacer esto. Dependiendo de la comodidad y práctica del desarrollador puede hacerse que el archivo PMML haga el cambio, o puede hacerse desde Spark mismo.

In [ ]:
!pip install pyspark==3.2.0 pyspark2pmml openscoring==0.5.0
!wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
!wget https://repo1.maven.org/maven2/org/jpmml/pmml-sparkml/2.2.0/pmml-sparkml-2.2.0.jar
!wget https://github.com/jpmml/jpmml-sparkml/releases/download/2.2.0/pmml-sparkml-example-executable-2.2.0.jar
!cp *.jar /usr/local/lib/python3.7/dist-packages/pyspark/jars
!wget https://github.com/openscoring/openscoring/releases/download/2.1.0/openscoring-server-executable-2.1.0.jar

In [ ]:
import os
from openscoring import Openscoring
from pyspark2pmml import PMMLBuilder

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql.types import StructType, DoubleType, StringType
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, udf, rand
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import (
    MinMaxScaler,
    VectorAssembler,
    OneHotEncoder,
    StringIndexer,
    IndexToString,
    StringIndexerModel,
    MinMaxScalerModel,
    SQLTransformer
)

In [ ]:
jars = [
  'pmml-sparkml-example-executable-2.2.0.jar',
  'pmml-sparkml-2.2.0.jar',
  '/content/pmml-sparkml-example-executable-2.2.0.jar',
  '/content/pmml-sparkml-2.2.0.jar'
]

joined_jars = ",".join(jars)
os.environ['PYSPARK_SUBMIT_ARGS'] = \
  '--packages org.jpmml:pmml-sparkml:2.0.0 ' + \
  f'--master local[*] --jars {joined_jars} ' + \
  'pyspark-shell'

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .config('spark.jars', joined_jars) \
    .appName("PMML, PFA y MLeap") \
    .getOrCreate()
sc = spark.sparkContext

In [ ]:
#Solucion